In [1]:
import os
import openai
from dotenv import load_dotenv
import sys
sys.path.append('../..')

# Obtén la ruta absoluta del archivo openai.env
env_path = '/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/openai.env'

load_dotenv(dotenv_path=env_path, override=True)


api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

In [2]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Leer el archivo de Excel
df = pd.read_excel('/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/openai/ejemplos.xlsx')

# Guardar el DataFrame en un archivo CSV
df = df.to_csv('/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/openai/ejemplos.csv', index=False)

loader = CSVLoader(file_path='/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/openai/ejemplos.csv')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(data)

embedding = OpenAIEmbeddings()

# Crear lista de textos desde documentos divididos
text_list = [doc.page_content for doc in splits]

# Realizar embedding en los textos
embeddings = embedding.embed_documents(text_list)

# Load embeddings into Chroma
db = Chroma.from_documents(splits, embedding)

In [3]:
import pandas as pd
import openai
import time
from update_names import obtener_nombre_ecommerce
from openai.error import OpenAIError


def procesar_proveedor(proveedor):
    path = f'/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/data/proveedor/{proveedor}/listos/{proveedor}.xlsx'

    # Lee el archivo xlsx y selecciona las columnas requeridas
    df = pd.read_excel(path, engine='openpyxl')
    df = df[['Identificador de URL', 'canal_Nombre', 'df2_Nombre','canal_Marca']]

    # Elimina las filas con valores NaN en las columnas 'Identificador de URL' y 'canal_Nombre'
    df = df.dropna(subset=['Identificador de URL', 'canal_Nombre','df2_Nombre'])

    # Crear una copia del DataFrame original
    df_original = df.copy()

    # Eliminar la columna 'Identificador de URL', 'canal_Marca' y 'canal_Nombre' del DataFrame de trabajo
    df = df.drop(['canal_Nombre', 'Identificador de URL'], axis=1)

    # # Lee el archivo de ejemplos
    # df_ejemplos = pd.read_excel('ejemplos.xlsx')
    # ejemplos = df_ejemplos.to_dict('records')
    # # Formatea los ejemplos para la entrada de la API de OpenAI
    # ejemplos_formateados = [f"{ejemplo['original']} -> '{ejemplo['corregido']}'" for ejemplo in ejemplos]
    # ejemplos_string = "\n".join(ejemplos_formateados)

    df['nombre_ecommerce'] = ''

    print(df.head())
    contador = 0

    for i, row in df.iterrows():
        # Definir el nombre del producto sin corregir
        query = row['df2_Nombre']

        # Realizar una búsqueda de similitud
        similar_docs = db.similarity_search(query, top_k=50, limit=50)

        # Aquí, similar_docs es una lista de documentos que son los más similares a 'query'.
        # Convierte estos a los ejemplos para la entrada de la API de OpenAI
        ejemplos = [doc.page_content for doc in similar_docs]
        ejemplos_formateados = [f"'{ejemplo}'" for ejemplo in ejemplos]
        ejemplos_string = "\n".join(ejemplos_formateados)

        marca = "" if pd.isna(row['canal_Marca']) else f". Marca del producto: '{row['canal_Marca']}'"
        messages = [
            {
                "role": "system", 
                "content": (
                    "Tienes la tarea de generar nombres atractivos para productos para el comercio "
                    "electrónico, siguiendo el formato: 'Marca - Nombre del producto - Cantidad - "
                    "Color - Otras características relevantes'. El nombre del producto debe ser descriptivo, "
                    "pero corto y simple. Evita el vocabulario técnico o palabras que puedan confundir al cliente."
                )
            },
            {
                "role": "user", 
                "content": f"""
                    Aquí están algunos ejemplos de nombres de productos renovados: 
                    {ejemplos_string}
                    Ahora, genera un nombre atractivo para el siguiente producto. Ten en cuenta que el 
                    nombre debe ser descriptivo pero a la vez corto y sencillo. Evita el vocabulario técnico 
                    o palabras que puedan confundir al cliente. Nombre del producto original: '{row['df2_Nombre']}'{marca}.
                """
            }
        ]
        while True:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-4",
                    messages=messages,
                    max_tokens=100,
                    temperature=0.6
                )
                # Si la llamada a la API tiene éxito, se interrumpe el bucle
                break
            except OpenAIError as e:
                # Si se produce un error, imprime el mensaje de error y luego espera un minuto antes de intentarlo de nuevo
                print(f"Error en la iteración {i}, el error es: {str(e)}")
                print("Esperando 20 segundos antes de reintentar.")
                time.sleep(20)




        # Asigna la respuesta del modelo a la columna correspondiente
        df.at[i, 'nombre_ecommerce'] = response['choices'][0]['message']['content'].strip()

        # Imprime la información de la iteración actual
        print(f"Iteración {i}")
        print(f"Original: {row['df2_Nombre']}")
        print(f"Nuevo: {response['choices'][0]['message']['content'].strip()}")
        print("------------------------")

        contador += 1
        if contador % 100 == 0:
            print("Tomando un receso de 30 segundos...")
            time.sleep(30)

    # Combina df_original y df en base al índice
    df_final = df_original.merge(df, left_index=True, right_index=True)

    # Selecciona solo las columnas 'Identificador de URL' y 'nombre_ecommerce'
    df_final = df_final[['Identificador de URL', 'nombre_ecommerce']]

    df_final['nombre_ecommerce'] = df_final['nombre_ecommerce'].apply(obtener_nombre_ecommerce)

    # Renombra la columna 'nombre_ecommerce' a 'Nombre'
    df_final = df_final.rename(columns={'nombre_ecommerce': 'Nombre'})

    # Guarda el DataFrame a un nuevo archivo .xlsx
    df_final.to_excel(f'{proveedor}_results.xlsx', index=False)


In [4]:
procesar_proveedor('algabo')

                       df2_Nombre  canal_Marca nombre_ecommerce
0  Vais limpiavidrios gatillo 500         Vais                 
1      Algabo quita c/ dosif. 120       Algabo                 
2   Kids shampoo sandía dulce 350  Algabo Kids                 
3          Algabo quita black 100       Algabo                 
4    Algabo quita xtreme prof 100       Algabo                 
Iteración 0
Original: Vais limpiavidrios gatillo 500
Nuevo: Vais - Limpiador de Vidrios con Gatillo - 500 ml
------------------------
Iteración 1
Original: Algabo quita c/ dosif. 120
Nuevo: 'Algabo - Quitaesmalte con Dosificador - 120 ml'
------------------------
Iteración 2
Original: Kids shampoo sandía dulce 350
Nuevo: 'Algabo Kids - Shampoo Sandía Dulce - 350 ml'
------------------------
Iteración 3
Original: Algabo quita black 100
Nuevo: 'Algabo - Quitaesmalte Negro - 100 ml'
------------------------
Iteración 4
Original: Algabo quita xtreme prof 100
Nuevo: 'Algabo - Quitaesmalte Xtreme Profesional - 100 

KeyboardInterrupt: 